In [1]:
import pandas as pd
import os

### Import CSV

Import full file.

In [2]:
file_path = '../data/income-data/el20140713.csv'

In [3]:
full_df = pd.read_csv(file_path, encoding='Latin-1')

### Wrangle

To wrangle, we need indexes.

- **Pay Period Info**: 'Year, Pay Period'
- **Earnings**: 'Earnings and Deductions'
- **Leave**: 'Year-to-Date Leave Status	'
- **Agency Contributions**: 'Agency Contributions to Employee Benefits this Pay Period'

Separate out each piece.

In [4]:
names = ['Year, Pay Period', 'Earnings and Deductions', 'Year-to-Date Leave Status',
         'Agency Contributions to Employee Benefits this Pay Period']

indexes = full_df.index[full_df['Gregory Barbieri'].isin(names)].tolist()

In [5]:
for item, index in enumerate(indexes):
    start = index
    
    if item < len(indexes)-1:
        end=indexes[item+1]

    if item == 0:
        pay_period_info = full_df.iloc[start:end].reset_index(drop=True)
    elif item == 1:
        earnings = full_df.iloc[start:end].reset_index(drop=True)
    elif item == 2:
        leave = full_df.iloc[start:end].reset_index(drop=True)
    elif item == 3:
        agency_contributions = full_df.iloc[start:].reset_index(drop=True)

Wrangle Pay-period Info.

In [6]:
columns = pay_period_info.columns.tolist()

rename_cols = ['pay_period','agency','pay_plan','salary',
               'scd','na_col_1','na_col_2','na_col_3']

rename_dict = {}

for index, column in enumerate(columns):
    rename_dict[column] = rename_cols[index]
    
pay_period_info.rename(columns=rename_dict, inplace=True)    

In [7]:
pay_period_info = pay_period_info.loc[pay_period_info['pay_period'].isnull() == False]
pay_period_info['pay_period_range'] = pay_period_info['pay_period'].shift(-1)
pay_period_info = pay_period_info.loc[(pay_period_info['agency'].isnull() == False) &
                                      (pay_period_info['salary'] != 'Salary')]
pay_period_info['pay_period'] = pay_period_info['pay_period'].str.replace(u'\xa0', u' ').str.replace(' ','')
pay_period_info = pay_period_info.drop(columns=['na_col_1','na_col_2','na_col_3']).reset_index(drop=True)

Wrangle earnings data.

In [8]:
columns = earnings.columns.tolist()

rename_cols = ['code','description','hours_pp','hours_ytd',
               'amount_pp','amount_ytd','na_col_1','na_col_2']

rename_dict = {}

for index, column in enumerate(columns):
    rename_dict[column] = rename_cols[index]

earnings.rename(columns=rename_dict, inplace=True)

In [9]:
earnings = earnings.loc[(earnings['code'] != '**') &
                        (earnings['code'].isnull() == False) &
                        (earnings['code'] != 'Code') &
                        (earnings['description'].isnull() == False)]

earnings['pay_period'] = pay_period_info['pay_period'][0]

earnings = earnings.drop(columns=['na_col_1','na_col_2']).reset_index(drop=True)

Wrangle Leave Usage and Accrual

In [11]:
columns = leave.columns.tolist()

rename_cols = ['type','na_col_1','accured','used',
               'balance','projected_uol','na_col_2','na_col_3']

rename_dict = {}

for index, column in enumerate(columns):
    rename_dict[column] = rename_cols[index]

leave.rename(columns=rename_dict, inplace=True)

In [12]:
leave = leave.loc[(leave['type'].isnull() == False) &
                  (leave['type'] != 'Type') &
                  (leave['accured'].isnull() == False)]

leave['pay_period'] = pay_period_info['pay_period'][0]

leave = leave.drop(columns=['na_col_1','na_col_2','na_col_3']).reset_index(drop=True)

Wrangle Agency Contributions

In [14]:
columns = agency_contributions.columns.tolist()

rename_cols = ['contribution','amount','na_col_1','na_col_2','na_col_3','na_col_4','na_col_5','na_col_6']

rename_dict = {}

for index, column in enumerate(columns):
    rename_dict[column] = rename_cols[index]

agency_contributions.rename(columns=rename_dict, inplace=True)

In [15]:
agency_contributions = agency_contributions.loc[agency_contributions['amount'].isnull() == False]

agency_contributions['contribution'] = agency_contributions['contribution'].str.replace("*","")

agency_contributions['pay_period'] = pay_period_info['pay_period'][0]

agency_contributions = agency_contributions.drop(columns=['na_col_1','na_col_2','na_col_3','na_col_4',
                                                          'na_col_5','na_col_6']).reset_index(drop=True)